In [15]:
pip install Faker
pip install openai

  Obtaining dependency information for Faker from https://files.pythonhosted.org/packages/70/2a/479c5479bcc3ea7d82e34d3b4541dc456a29192ab5298af4887055f7f816/Faker-25.0.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 13.6 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


# Genereer namen, tijdstippen en zorgverlener IDs

In [2]:
import pandas as pd
from faker import Faker
import random
from datetime import datetime

# Specify size parameters
n_names = 1
n_per_name = 3


# Setup faker for Dutch names and data
fake = Faker('nl_NL')
Faker.seed(0)
random.seed(0)

# Generate client names
names = [fake.name() for _ in range(n_names)]

# Each name appears three times
client_names = [name for name in names for _ in range(n_per_name)]

# Generate random timestamps starting from 2020
random_dates = pd.to_datetime(
    [fake.date_time_between(start_date='-3y', end_date='now') for _ in range(n_names * n_per_name)]
).sort_values().strftime('%Y-%m-%d %H:%M')

# Randomly assign zorgverlener IDs
zorgverlener_ids = [f"ZV10{random.randint(1,5)}" for _ in range(n_names * n_per_name)]

# Creating the DataFrame
data = {
    "Cliëntnaam": client_names,
    "Tijdstip": random_dates,
    "Zorgverlener ID": zorgverlener_ids
}

# Convert to DataFrame
df = pd.DataFrame(data)

# Shuffle client names to ensure they are mixed up and then sort by 'Tijdstip'
random.shuffle(df['Cliëntnaam'])
df_sorted = df.sort_values(by='Tijdstip')

# Display the DataFrame
print(df_sorted)


          Cliëntnaam          Tijdstip Zorgverlener ID
0  Noortje van Boven  2021-08-30 13:59           ZV104
1  Noortje van Boven  2022-08-04 21:44           ZV104
2  Noortje van Boven  2023-08-31 22:19           ZV101


## Genereer teksten met behulp van GPT

In [3]:
from openai import OpenAI

client = OpenAI(
    # This is the default and can be omitted
    api_key = ""
)

# Assuming you have a specific entity and sector to check

def rapport_generator(name):
    response = client.chat.completions.create(
        messages=[
        {"role": "system", "content": "Je bent een generator voor teksten. Maak de tekst gemiddeld 50 woorden lang. Het moet zijn alsof een persoonlijk begeleider een dagelijkse rapportage schrijft over iemand met een verstandelijke beperking. Zorg er voor dat er nepnamen, nep adressen, nep e-mail adressen en nep telefoon nummers in de rapportage voorkomen. Het stukje tekst moet gaan over de dag van de persoon met de verstandelijke beperking. "},
        {"role": "user", "content": name}
        ],
        max_tokens = 200,
        model="gpt-3.5-turbo",
    )
    result = response.choices[0].message.content
    print(name)
    return result

In [4]:
rapport_generator("jan jansen")

APIConnectionError: Connection error.

# Maak de dataset en exporteer

In [26]:
df_sorted['rapport'] = df_sorted['Cliëntnaam'].apply(rapport_generator)
print(df_sorted)

Liv Goedhart-Wildschut
Hendrik van de Ven
Hendrik van de Ven
Lizzy Molenaar
Tom Verheij
Liv Goedhart-Wildschut
Hailey Mathieu
Hendrik van de Ven
Jenna Stoffel-van Oosten
Jari Glasses
Tom Verheij
Hailey Mathieu
Tom Verheij
Hailey Mathieu
Lizzy Molenaar
Ivan Janse
Noortje van Boven
Noortje van Boven
Liv Goedhart-Wildschut
Noortje van Boven
Jenna Stoffel-van Oosten
Milan Prinsen
Ivan Janse
Milan Prinsen
Milan Prinsen
Jenna Stoffel-van Oosten
Jari Glasses
Jari Glasses
Lizzy Molenaar
Ivan Janse
                  Cliëntnaam          Tijdstip Zorgverlener ID  \
0     Liv Goedhart-Wildschut  2021-05-04 04:48           ZV104   
1         Hendrik van de Ven  2021-05-18 08:00           ZV104   
2         Hendrik van de Ven  2021-07-30 02:15           ZV101   
3             Lizzy Molenaar  2021-08-30 10:40           ZV103   
4                Tom Verheij  2021-11-28 07:17           ZV105   
5     Liv Goedhart-Wildschut  2022-01-19 09:41           ZV104   
6             Hailey Mathieu  2022-01-25 04

In [30]:

# Gebruik datetime om de huidige datum en tijd te verkrijgen en formatteren
current_time = datetime.now().strftime('%Y%m%d_%H%M%S')

# Maak een pad voor het opslaan van het bestand, inclusief de huidige datum en tijd
csv_path = f'ECD_dummy_dataset_{current_time}.csv'

# Exporteer het DataFrame naar een CSV-bestand zonder index
df_sorted.to_csv(csv_path, index=False)

# Print het pad waar het bestand is opgeslagen
print(f'Bestand opgeslagen op: {csv_path}')

Bestand opgeslagen op: ECD_dummy_dataset_20240502_161537.csv
